In [6]:
!pip install torchinfo

In [2]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor
from tqdm import tqdm

import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

/home/cdsw/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-04 07:59:53,829	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-04 07:59:58,174	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-04-04 07:59:59,095	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
from torchinfo import summary
model = NeuralNetwork()
input_size = (1, 28, 28)  # Example input size for a single image
summary(model, input_size=input_size)

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [1, 10]                   --
├─Flatten: 1-1                           [1, 784]                  --
├─Sequential: 1-2                        [1, 10]                   --
│    └─Linear: 2-1                       [1, 512]                  401,920
│    └─ReLU: 2-2                         [1, 512]                  --
│    └─Dropout: 2-3                      [1, 512]                  --
│    └─Linear: 2-4                       [1, 512]                  262,656
│    └─ReLU: 2-5                         [1, 512]                  --
│    └─Dropout: 2-6                      [1, 512]                  --
│    └─Linear: 2-7                       [1, 10]                   5,130
│    └─ReLU: 2-8                         [1, 10]                   --
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
Total mult-adds (M): 0.67
Input size (MB): 0.00
Forward/backward pass 

## Connecting to an existing Ray Cluster

In [16]:
import ray
import json
import os

if os.path.exists("ray_address.json"):
    with open("ray_address.json", "r") as f:
        connection_info = json.load(f)
        ray_address = connection_info["ray_address"]
        dashboard_url = connection_info["dashboard_url"]

        ray.init(address=ray_address, allow_multiple=True)
        print(f"✅ Connected to an existing Ray cluster at {ray_address}")
        print(f"📊 Ray Dashboard: {dashboard_url}")
else:
    print("❌ Ray cluster not initialized. Please start it first.")

2025-04-07 12:47:51,283	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver


✅ Connected to an existing Ray cluster at ray://10.254.5.113:10001
📊 Ray Dashboard: https://734nix0zkpcgc133.goes-ocp-cml.apps.field-team-ocp-01.kcloud.cloudera.com/


## Launching Tensor Board

In [14]:
# Launch Tensor Board in the background

import subprocess
import os

# Define logdir and port
logdir = "./ray_results/tb_logs"
port = os.environ.get("CDSW_PUBLIC_PORT")

# Launch TensorBoard in the background
tensorboard_proc = subprocess.Popen(
    ["tensorboard", f"--logdir={logdir}", f"--port={port}"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

engine_id = os.environ.get("CDSW_ENGINE_ID", "<engine_id_not_found>")
domain = os.environ.get("CDSW_DOMAIN", "<domain_not_found>")

url = f"https://public-{engine_id}.{domain}"


print(f"✅ TensorBoard launched in background on port {port}")
print(f"Access it at: {url}")




✅ TensorBoard launched in background on port 8080
Access it at: https://public-ehtmlmkedggjmarl.goes-ocp-cml.apps.field-team-ocp-01.kcloud.cloudera.com


Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2025-04-07 12:28:13,649	WARNING dataclient.py:403 -- Encountered connection issues in the data channel. Attempting to reconnect.
2025-04-07 12:28:43,859	WARNING dataclient.py:410 -- Failed to reconnect the data channel


In [12]:
import os





print(url)

https://public-ehtmlmkedggjmarl.goes-ocp-cml.apps.field-team-ocp-01.kcloud.cloudera.com
